# Universal Thermal Comfort Index (UTCI) classes
This notebook calculates the number of nights per year in each UTCI class, based on the UTCI hourly data available at the Copernicus climate data store, in NetCDF format. The values were calculated for the nights: The maximum temperatures between 18h-6h were used.
- UTCI definition: For any combination of air temperature, wind, radiation and humidity, the Universal thermal climate index is defined as the air temperature of a reference outdoor environment that would elicit in the human body the same physiological model’s response (sweat production, shivering, skin wettedness, skin blood flow and rectal, mean skin and face temperatures) as the actual environment.
- Data source: https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview
- UTCI classes:

| UTCI value \[°C\] | UTCI class              |
|----------------|-------------------------|
| >46            | Extreme Heat Stress     |
| 38-46          | Very Strong Heat Stress |
| 32-38          | Strong Heat Stress      |
| 26-32          | Moderate Heat Stress    |
| 9-26           | No Thermal Stress       |
| 0-9            | Slight Cold Stress      |
| -13-0          | Moderate Cold Stress    |
| -27--13        | Strong Cold Stress      |
| -40--27        | Very Strong Cold Stress |
| <-40           | Extreme Cold Stress     |

## Workflow
1. Download yearly data from CDS. Unzip the files in `utci_hourly_path/<year>`. There is one NetCDF file per day
2. Subset and re-chunk the data into one file and save it in `utci_yearly_path`
3. Compute the classes
4. Save results to database

## Important note
Since the data is in NetCDF format, which is not cloud-optimized, it is highly recommended to download the data locally and run this notebook in a your local machine.

TODO check new cds apis

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from datetime import datetime, timedelta

year = '2018'
utci_hourly_path = f'N:/C2205_FAIRiCUBE/f02_data/d006_climate_data/f01_thermal_comfort_index/{year}'
utci_yearly_path = f'N:/C2205_FAIRiCUBE/f02_data/d006_climate_data/f01_thermal_comfort_index/yearly'


## Open hourly data
Open all data for one year, convert from Kelvin to Celsius

In [2]:
data_one_year_df = xr.open_mfdataset(utci_hourly_path+"/*.nc")

utci = data_one_year_df.utci -273.15
utci.attrs = data_one_year_df.utci.attrs
utci.attrs["units"] = "C"

In [3]:
utci

<xarray.DataArray 'utci' (time: 8760, lat: 601, lon: 1440)>
dask.array<sub, shape=(8760, 601, 1440), dtype=float32, chunksize=(24, 601, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 ... 2018-12-31T23:00:00
  * lon      (lon) float64 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -59.25 -59.5 -59.75 -60.0
Attributes:
    code:     167
    table:    128
    units:    C

## Get cities and coordinates and filter UTCI data

In [ ]:
# get df cities
# download URAU_LB_2021_4326 from Eurostat GISCO first
cities_path = "./data/eu_cities_atlas/URAU_LB_2021_4326.geojson"
load_cities_df = gpd.read_file(cities_path)
load_cities_df.rename(columns={"URAU_CODE": "city_code"}, inplace=True)
# extract x,y coordinates from geometry
load_cities_df["_wgs84x"] = load_cities_df.geometry.x
load_cities_df["_wgs84y"] = load_cities_df.geometry.y
    
city_center_df_r = load_cities_df#[60:62] #use this to create dataset subset

# get city coordinates
# lonlat_list =[["NL005C", 4.640960, 52.113299], ["NL006C", 5.384670, 52.173656], ["NL007C", 5.921886, 52.189884]]

lon_list = city_center_df_r["_wgs84x"].values.tolist()
lat_list = city_center_df_r["_wgs84y"].values.tolist()
city_list = city_center_df_r["urau_code"].values.tolist()
target_lon = xr.DataArray(lon_list, dims="city", coords={"city": city_list})
target_lat = xr.DataArray(lat_list, dims="city", coords={"city": city_list})
time_zone_offset = xr.DataArray(city_center_df_r['time_zone_offset'], dims="city", coords={"city": city_list})

## next filter dataframe by city:

data_one_year_df_cities = utci.sel(
    lon=target_lon, 
    lat=target_lat, method="ffill")

### Save UTCI data subset to one file
This speeds up further computations because avoids opening all daily NetCDF files for the entire data extent

In [6]:
data_one_year_df_cities_tz = xr.merge([data_one_year_df_cities,time_zone_offset])
# save city dataset
data_one_year_df_cities_tz.to_netcdf(path = utci_yearly_path+f"/{year}_cities.nc")
data_one_year_df_cities_tz

<xarray.Dataset>
Dimensions:           (time: 8760, city: 729)
Coordinates:
  * time              (time) datetime64[ns] 2018-01-01 ... 2018-12-31T23:00:00
    lon               (city) float64 -3.0 8.5 23.5 14.75 ... 14.25 24.75 17.0
    lat               (city) float64 43.5 47.25 43.5 41.0 ... 46.25 45.0 48.25
  * city              (city) <U6 'ES019C' 'CH012C' ... 'RO028C' 'SK001C'
Data variables:
    utci              (time, city) float32 dask.array<chunksize=(24, 729), meta=np.ndarray>
    time_zone_offset  (city) int64 1 1 2 1 2 1 1 1 2 1 1 ... 1 2 1 1 1 2 1 1 2 1
Attributes:
    code:     167
    table:    128
    units:    C

### Load data and convert to Dask DataFrame

In [15]:
load_data = xr.open_mfdataset(utci_yearly_path+f"/{year}_cities.nc")
load_data_df = load_data.chunk({'city': 100, 'time': -1}).to_dask_dataframe()
load_data_df

,time,city,lon,lat,utci,time_zone_offset
npartitions=8,,,,,,
0,datetime64[ns],object,float64,float64,float32,int64
798255,...,...,...,...,...,...
...,...,...,...,...,...,...
5587785,...,...,...,...,...,...
6386039,...,...,...,...,...,...


## Compute nighttime max UTCI 
### Helper functions

In [16]:
def daytime(x):
    time_local = x.time+timedelta(hours=x.time_zone_offset)
    if(time_local.hour >= 6 and time_local.hour < 18):
        return 1
    else:
        return 0

def night_date(x):
    # assign previous day date to nighttime hours [0..end_night]
    # first adjust to local time
    time_local = x.time+timedelta(hours=x.time_zone_offset)
    if(time_local.hour < 6):
        # date = x.time - timedelta(days=1)
        return x.time - timedelta(days=1)
    else:
        return x.time
    
def classify_utci_df(x):
    utci_value = x.utci
    if utci_value > 46:           return "utci_class_extreme_heat_stress"
    elif 38 <= utci_value <= 46:  return "utci_class_very_strong_heat_stress"
    elif 32 <= utci_value < 38:   return "utci_class_strong_heat_stress"
    elif 26 <= utci_value < 32:   return "utci_class_moderate_heat_stress"
    elif 9 <= utci_value < 26:    return "utci_class_no_thermal_stress"
    elif 0 <= utci_value < 9:     return "utci_class_slight_cold_stress"
    elif -13 <= utci_value < 0:   return "utci_class_moderate_cold_stress"
    elif -27 <= utci_value < -13: return "utci_class_strong_cold_stress"
    elif -40 <= utci_value < -27: return "utci_class_very_strong_cold_stress"
    else:                         return "utci_class_extreme_cold_stress"

### Start Dask cluster

In [17]:
from dask.distributed import Client, performance_report
client = Client()  # Connect to distributed cluster and override default
client

c:\ProgramData\Anaconda3\envs\geospatial2\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59750 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:59750/status,
Dashboard: http://127.0.0.1:59750/status,Workers: 4
Total threads: 8,Total memory: 63.94 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59751,Workers: 4
Dashboard: http://127.0.0.1:59750/status,Total threads: 8
Started: Just now,Total memory: 63.94 GiB
Comm: tcp://127.0.0.1:59780,Total threads: 2
Dashboard: http://127.0.0.1:59781/status,Memory: 15.99 GiB
Nanny: tcp://127.0.0.1:59755,


### Compute

In [ ]:
load_data_df['daytime'] = load_data_df.apply(daytime, axis=1, meta=(None, 'int'))
load_data_df['time_shifted'] = load_data_df.apply(night_date, axis=1, meta=(None, 'datetime64[ns]'))
load_data_df['date'] = load_data_df.time_shifted.dt.date
utci_max = load_data_df.groupby(['city', 'daytime', 'date']).max()
utci_max['utci_class'] = utci_max.apply(classify_utci_df, axis=1, meta=(None, 'string'))
with performance_report(filename='utci_classes_2018_df_v2.html'):
    utci_max_c = utci_max.compute()
utci_max_c.reset_index(inplace=True)
client.close()

In [19]:
df = utci_max_c[utci_max_c.daytime == 0][['city', 'utci', 'utci_class']]
df_count = df.groupby(['city', 'utci_class']).count()
df_count

utci
city   utci_class                              
AT001C utci_class_moderate_cold_stress       84
       utci_class_moderate_heat_stress       45
       utci_class_no_thermal_stress         142
       utci_class_slight_cold_stress         81
       utci_class_strong_cold_stress         12
...                                         ...
SK008C utci_class_moderate_heat_stress       27
       utci_class_no_thermal_stress         153
       utci_class_slight_cold_stress         78
       utci_class_strong_cold_stress         13
       utci_class_very_strong_cold_stress     1

[4136 rows x 1 columns]

## Save indicator to csv

In [ ]:
##################################
## cu_* tables mandatory columns:
# city_code,
# parameter, 
# parameter_id, 
# parameter_value, 
# year, 
# city_code_version, 
# lineage, 
# datasource

df_count.reset_index(inplace=True)
df_count.rename(columns={'city': 'city_code',
                        'utci': 'parameter_value',
                        'utci_class': 'parameter_id'}, inplace=True)

# new columns:

df_count['year'] = year
df_count['city_code_version'] = 'ua_2021'
df_count['parameter'] = 'Number of nights per year in UTCI class: ' + df_count['parameter_id']
df_count['lineage'] = 'The night hours were first extracted from the hourly values: from locally 18h to locally 6h. In the next step, the UTCI temps were calculated within the NIGHT and transferred to UTCI classes. The number of days (nights) for each class was then calculated for each year.'
df_count['datasource']= 'https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview'
df_count.to_csv('data/UTCI_night_count_2018.csv')
df_count.head()

,city_code,parameter_id,parameter_value,year,city_code_version,parameter,lineage,datasource
0,AT001C,utci_class_moderate_cold_stress,84,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
1,AT001C,utci_class_moderate_heat_stress,45,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
2,AT001C,utci_class_no_thermal_stress,142,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
3,AT001C,utci_class_slight_cold_stress,81,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
4,AT001C,utci_class_strong_cold_stress,12,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
...,...,...,...,...,...,...,...,...
4131,SK008C,utci_class_moderate_heat_stress,27,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
4132,SK008C,utci_class_no_thermal_stress,153,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
4133,SK008C,utci_class_slight_cold_stress,78,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
4134,SK008C,utci_class_strong_cold_stress,13,2018,ua_2021,Number of nights per year in UTCI class: utci_...,The night hours were first extracted from the ...,https://cds.climate.copernicus.eu/cdsapp#!/dat...
